# 設置環境並載入套件

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/CorpusforDDL_compute/corpus_compute')
print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/CorpusforDDL_compute/corpus_compute


In [2]:
import pandas as pd
import re
import unicodedata
import pickle
from collections import Counter

In [3]:
# 載入事先寫好的 functions
from compute_parameters import *

In [4]:
%%capture
pip install -U ckip-transformers

In [5]:
import ckip_transformers
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
ws_driver = CkipWordSegmenter(device=0)
pos_driver = CkipPosTagger(device=0)

Downloading:   0%|          | 0.00/804 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/407M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/407M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

# 載入語料

In [6]:
# 讀入 Mandarin Daily News (小朋友作文)
essay_path = '../CorpusforDDL/MadarinDailyNews_from Zeng Yuanxian'
all_files = os.listdir(essay_path)

essay_corpus = []
for f in all_files:
  with open(f'{essay_path}/{f}', 'rb') as f:
      lines = f.readlines()
      essay_corpus.append(lines)

In [7]:
essay_corpus_decoded = []

for text in essay_corpus:
  text_decoded = []
  not_decoded = []
  for string in text:
    try:
      text = string.decode('utf-8')
      text_decoded.append(text)
    except:
      not_decoded.append(string)
  essay_corpus_decoded.append(text_decoded)

In [8]:
# 前處理
def preprocess_essay(string):

  clean_string = unicodedata.normalize('NFKC', string) # 全形轉半形
  clean_string = clean_string.translate(str.maketrans({',': '，', 
                                                       '!': '！', 
                                                       '?': '？'})) # ，！？改回全形
  clean_string = re.sub(r'.+\t.+\t.+\t', '', string) # 移除日期、題目、作者名
  clean_string = re.sub(r'。。', '', clean_string) # 移除多餘句號
  clean_string = re.sub(r'\r|\n', '', clean_string) # 移除換行符號

  return clean_string

In [9]:
essay_corpus_preprocessed = []

for text in essay_corpus_decoded:
  preprocessed = [preprocess_essay(string) for string in text]
  joined = ''.join(preprocessed)
  split_1 = re.split(r'(?<=。」|！」|？」)', joined) # 先用 。」 ！」 ？」 分隔
  split_2 = [re.split(r'(?<=[。！？])(?!」)', x) for x in split_1] # 再用 。！？ 分隔
  split = [item for sublist in split_2 for item in sublist] # 將 list of list 攤平
  essay_corpus_preprocessed.append(split)

# `ckip-transformers` 斷詞 + pos tag

In [11]:
essay_test = essay_corpus_preprocessed[0]
essay_test[0]

'新的開始，從心開始！'

In [12]:
# 斷詞
essay_ws  = ws_driver(essay_test)
with open('../pkl_files/essay_ws.pkl', 'wb') as f:
    pickle.dump(essay_ws, f)

Inference: 100%|██████████| 8/8 [00:58<00:00,  7.29s/it]


In [13]:
# pos tag
essay_pos = pos_driver(essay_ws)
with open('../pkl_files/essay_pos.pkl', 'wb') as f:
    pickle.dump(essay_pos, f)

Inference: 100%|██████████| 27/27 [03:04<00:00,  6.83s/it]


In [24]:
with open('../pkl_files/essay_ws.pkl', 'rb') as f:
  essay_ws_punc = pickle.load(f)

# with open('../pkl_files/essay_pos.pkl', 'rb') as f:
#   essay_pos = pickle.load(f)

## 語料格式準備 - 1
斷詞 + pos tag

In [14]:
essay_ws_pos = []

for sent_ws, sent_pos in zip(essay_ws, essay_pos):
  sent_ws_pos = []
  for ws, pos in zip(sent_ws, sent_pos):
    ws_pos = f"{ws}({pos})"
    sent_ws_pos.append(ws_pos)
  essay_ws_pos.append(' '.join(sent_ws_pos))

In [15]:
essay_ws_pos[0]

'新(VH) 的(DE) 開始(Nv) ，(COMMACATEGORY) 從(P) 心(Na) 開始(VH) ！(EXCLAMATIONCATEGORY)'

## 語料格式準備 - 2
斷好詞，無標點符號

In [25]:
essay_ws = []

for sentence in essay_ws_punc:
  clean_sent = []
  for word in sentence:
    if re.match(r'[A-z0-9\w]+', word):
      clean_sent.append(word)
  essay_ws.append(clean_sent)

In [26]:
essay_ws[0]

['新', '的', '開始', '從', '心', '開始']

## 語料格式準備 - 3
完整句子，含標點符號

In [27]:
essay_text = []
for sent in essay_ws_pos:
  res = re.sub(r'\([A-z0-9_]+\)', '', sent)
  res = res.replace(' ', '')
  essay_text.append(res)

essay_text[0]

'新的開始，從心開始！'

# 句子長度

In [28]:
sentence_length = [len(sentence) for sentence in essay_ws]
sentence_length[:10]

[6, 24, 21, 10, 18, 27, 10, 31, 24, 10]

# 高頻詞與低頻詞

In [29]:
high_low_freq = [get_high_low_freq(sentence) for sentence in essay_ws]
high_low_freq[:10]

[{'High': 6},
 {'High': 23, 'Low': 1},
 {'High': 21},
 {'High': 9, 'Low': 1},
 {'Low': 1, 'High': 17},
 {'Low': 3, 'High': 24},
 {'High': 9, 'Low': 1},
 {'High': 30, 'Low': 1},
 {'High': 24},
 {'High': 9, 'Low': 1}]

# 詞頻

In [30]:
word_freq = [get_word_freq(sentence) for sentence in essay_ws]
word_freq[0]

'新(212453) 的(16029838) 開始(245562) 從(496546) 心(180243) 開始(245562)'

# 詞彙等級

In [31]:
word_level = [get_word_level(sentence) for sentence in essay_ws]
word_level[0]

{'第1級': 2, '第2級': 3, '第1*級': 1}

# 詞彙長度

In [32]:
long_word_count = [get_long_word_count(sentence) for sentence in essay_ws]
long_word_count[:10]

[0, 0, 2, 0, 1, 2, 0, 0, 1, 3]

# 完整的句子

In [33]:
is_complete_sentence = [get_complete_sentence(sentence) for sentence in essay_ws_pos]
is_complete_sentence[:10]

['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y']

# 完整的語境

In [34]:
is_complete_context = [get_complete_context(x) for x in essay_ws_pos]
is_complete_context[:10]

['N', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'N']

# Black list

In [35]:
is_blacklist = [get_blacklist(x) for x in essay_text]
is_blacklist[:10]

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

# Grey list

In [36]:
is_greylist = [get_greylist(x) for x in essay_ws_pos]
is_greylist[:10] 

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

# Make dataframe

In [37]:
essay_df = pd.DataFrame({'sentence': essay_text,
                        'sentence_preprocessed': [' '.join(sent) for sent in essay_ws],
                        'sentence_length': sentence_length,
                        'word_freq': word_freq,
                        'high_low_freq': high_low_freq,
                        'word_level': word_level,
                        'long_word_count': long_word_count,
                        'is_complete_sentence': is_complete_sentence,
                        'is_complete_context': is_complete_context,
                        'is_blacklist': is_blacklist,
                        'is_greylist': is_greylist})

In [38]:
essay_df = essay_df[essay_df['sentence_length']>0]
essay_df

,sentence,sentence_preprocessed,sentence_length,word_freq,high_low_freq,word_level,long_word_count,is_complete_sentence,is_complete_context,is_blacklist,is_greylist
0,新的開始，從心開始！,新 的 開始 從 心 開始,6,新(212453) 的(16029838) 開始(245562) 從(496546) 心(1...,{'High': 6},"{'第1級': 2, '第2級': 3, '第1*級': 1}",0,Y,N,N,N
1,我要改掉自己的壞脾氣，我常為了芝麻綠豆大的小事而生氣，實在是太不應該了。,我 要 改掉 自己 的 壞 脾氣 我 常 為了 芝麻 綠豆 大 的 小 事 而 生氣 實在 ...,24,我(3033032) 要(851264) 改掉(785) 自己(744739) 的(1602...,"{'High': 23, 'Low': 1}","{'第1級': 10, 'Unknown': 3, '第1*級': 4, '第2*級': 2...",0,Y,Y,N,N
2,從現在開始，我要讓自己更有肚量，有話好好說，笑口常開，讓我的人緣越來越好。,從 現在 開始 我 要 讓 自己 更 有 肚量 有 話 好好 說 笑口常開 讓 我 的 人緣...,21,從(496546) 現在(218744) 開始(245562) 我(3033032) 要(8...,{'High': 21},"{'第1*級': 2, '第1級': 9, '第2級': 3, 'Unknown': 6, ...",2,Y,N,N,N
3,媽媽帶哥哥和我一起去菜園捉菜蟲。,媽媽 帶 哥哥 和 我 一起 去 菜園 捉 菜蟲,10,媽媽(70492) 帶(105786) 哥哥(18973) 和(1338337) 我(303...,"{'High': 9, 'Low': 1}","{'第1級': 6, 'Unknown': 3, '第5級': 1}",0,Y,Y,N,N
4,菜蟲吃的是菜，所以身體的保護色是綠色的，要仔細檢查才能發現。,菜蟲 吃 的 是 菜 所以 身體 的 保護色 是 綠色 的 要 仔細 檢查 才 能 發現,18,菜蟲(25) 吃(141660) 的(16029838) 是(4642407) 菜(1788...,"{'Low': 1, 'High': 17}","{'Unknown': 3, '第1級': 7, '第1*級': 4, '第4*級': 1,...",1,Y,Y,N,N
...,...,...,...,...,...,...,...,...,...,...,...
2034,我最喜歡他做的蝴蝶，老板用種子當翅膀，用竹子做觸角和支架。,我 最 喜歡 他 做 的 蝴蝶 老板 用 種子 當 翅膀 用 竹子 做 觸角 和 支架,18,我(3033032) 最(421680) 喜歡(113753) 他(2487930) 做(4...,{'High': 18},"{'第1級': 8, '第1*級': 1, '第5級': 3, '第2級': 1, 'Unk...",0,Y,Y,N,N
2035,一開始，我很懷疑直直的竹子怎麼可能做出彎曲的觸角？,一 開始 我 很 懷疑 直直 的 竹子 怎麼 可能 做出 彎曲 的 觸角,14,一(3701681) 開始(245562) 我(3033032) 很(715675) 懷疑(...,{'High': 14},"{'第1級': 7, '第2級': 1, '第4*級': 1, 'Unknown': 2, ...",0,Y,N,N,N
2036,沒想到，老板拿出打火機，在竹子旁燒烤，竹子就彎成想要的角度和捲度，真神奇！,沒想到 老板 拿出 打火機 在 竹子 旁 燒烤 竹子 就 彎成 想要 的 角度 和 捲度 真 神奇,18,沒想到(13287) 老板(848) 拿出(14551) 打火機(645) 在(328423...,"{'High': 17, 'Low': 1}","{'第3*級': 1, '第2級': 1, '第4級': 3, 'Unknown': 6, ...",2,Y,Y,N,N
2037,生活藝術除了要有美感，還需要有靈活的頭腦，運用生活經驗和智慧，就地取材。,生活 藝術 除了 要 有 美感 還 需要 有 靈活 的 頭腦 運用 生活 經驗 和 智慧 就...,19,生活(213189) 藝術(80645) 除了(75964) 要(851264) 有(221...,{'High': 19},"{'第3級': 2, '第4級': 1, '第3*級': 1, '第1級': 4, '第5級...",0,Y,Y,N,N


In [40]:
essay_df.to_csv('../results/children_essay/essay_parameters_example.csv', index = False)

## Concordance

In [41]:
from nltk.text import Text

In [42]:
target_words = ['難得', '畢竟', '的確', '難免', '總是', '有助於']
corpus = [item for sublist in essay_ws for item in sublist]
text = Text(corpus)
dfs = []

for word in target_words:

  con_list = text.concordance_list(word)
  right_word = [x.right[0] for x in con_list]
  left_word = [x.left[-1] for x in con_list]
  context = [x.left + [word] + x.right for x in con_list]
  context = [' '.join(x) for x in context]

  df = pd.DataFrame({'left_word': left_word,
                     'target_word': word,
                     'right_word': right_word,
                     'context': context})
  dfs.append(df)

In [43]:
concordance_df = pd.concat(dfs)
concordance_df.to_csv('../results/children_essay/essay_concordance_df.csv', index = False)